In [1]:
# 기본
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from matplotlib import font_manager, rc
import platform
from tqdm import tqdm
import sklearn
from sklearn import linear_model
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
from sklearn.model_selection import train_test_split

# crawling
import requests
import lxml.html
import sqlite3
from pandas.io import sql
from bs4 import BeautifulSoup

# 한글 폰트 설정
if platform.system() == 'Windows':
# 윈도우인 경우
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)

In [2]:
final = pd.read_csv('MME_NanAlldrop_VisitAll7_2406.csv', encoding = 'euc-kr')

In [3]:
final

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED
0,A01,NIH1604001317,200209.0,2,7,43.0,50.0,153.5,70.0,14.3,209.0,1.0,88.0,90.0,1.00
1,A02,NIH1604001317,200411.0,2,7,45.0,52.4,153.4,69.8,16.1,224.0,1.0,100.0,98.0,1.30
2,A03,NIH1604001317,200611.0,2,7,47.0,50.6,153.0,70.2,15.0,177.0,1.0,106.0,100.0,1.10
3,A04,NIH1604001317,200812.0,2,7,49.0,51.4,153.5,79.5,15.6,166.0,1.0,116.0,112.0,1.09
4,A05,NIH1604001317,201007.0,2,7,51.0,52.2,153.0,71.4,15.0,189.0,1.0,110.0,112.0,1.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,2,7,47.0,62.2,164.7,74.0,18.3,227.0,1.0,90.0,90.0,0.94
16838,A04,NIH1604999643,200711.0,2,7,49.0,60.3,164.8,67.1,15.6,222.0,1.0,86.0,94.0,0.92
16839,A05,NIH1604999643,200909.0,2,7,51.0,60.1,165.0,70.2,16.7,219.0,1.0,88.0,90.0,0.84
16840,A06,NIH1604999643,201107.0,2,7,53.0,59.8,164.5,68.1,15.6,265.0,1.0,94.0,96.0,0.87


In [9]:
# SBP R,L 평균내서 새로운 칼럼으로 추가

final['SBP_AVG'] = (final['SBP_R'] + final['SBP_L']) / 2.0
final

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED,SBP_AVG
0,A01,NIH1604001317,200209.0,2,7,43.0,50.0,153.5,70.0,14.3,209.0,1.0,88.0,90.0,1.00,89.0
1,A02,NIH1604001317,200411.0,2,7,45.0,52.4,153.4,69.8,16.1,224.0,1.0,100.0,98.0,1.30,99.0
2,A03,NIH1604001317,200611.0,2,7,47.0,50.6,153.0,70.2,15.0,177.0,1.0,106.0,100.0,1.10,103.0
3,A04,NIH1604001317,200812.0,2,7,49.0,51.4,153.5,79.5,15.6,166.0,1.0,116.0,112.0,1.09,114.0
4,A05,NIH1604001317,201007.0,2,7,51.0,52.2,153.0,71.4,15.0,189.0,1.0,110.0,112.0,1.03,111.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,2,7,47.0,62.2,164.7,74.0,18.3,227.0,1.0,90.0,90.0,0.94,90.0
16838,A04,NIH1604999643,200711.0,2,7,49.0,60.3,164.8,67.1,15.6,222.0,1.0,86.0,94.0,0.92,90.0
16839,A05,NIH1604999643,200909.0,2,7,51.0,60.1,165.0,70.2,16.7,219.0,1.0,88.0,90.0,0.84,89.0
16840,A06,NIH1604999643,201107.0,2,7,53.0,59.8,164.5,68.1,15.6,265.0,1.0,94.0,96.0,0.87,95.0


In [11]:
final['eGFR'] = 0.0

In [12]:
# eGFR 계산 함수

def calculate_eGFR(df): 
    tmpdf = df.copy()
    for i in range(len(tmpdf)): 
        if tmpdf['SEX'][i] == 1: # if male
            if tmpdf['CREA_ADJUSTED'][i] <= 0.9: 
                tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-0.411))*(0.993**tmpdf['AGE'][i])
            else: 
                tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-1.209))*(0.993**tmpdf['AGE'][i])
        if tmpdf['SEX'][i] == 2: # if female[i]
            if tmpdf['CREA_ADJUSTED'][i] <= 0.7: 
                tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-0.329))*(0.993**tmpdf['AGE'][i])
            else: 
                tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-1.209))*(0.993**tmpdf['AGE'][i])
                
    return tmpdf

In [13]:
final = calculate_eGFR(final)
final

<ipython-input-12-e6f1b093cf7a>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGFR'][i] = 144*((tmpdf['CREA_ADJUSTED'][i]/0.7)**(-1.209))*(0.993**tmpdf['AGE'][i])
<ipython-input-12-e6f1b093cf7a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGFR'][i] = 141*((tmpdf['CREA_ADJUSTED'][i]/0.9)**(-0.411))*(0.993**tmpdf['AGE'][i])
<ipython-input-12-e6f1b093cf7a>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmpdf['eGF

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED,SBP_AVG,eGFR
0,A01,NIH1604001317,200209.0,2,7,43.0,50.0,153.5,70.0,14.3,209.0,1.0,88.0,90.0,1.00,89.0,69.167759
1,A02,NIH1604001317,200411.0,2,7,45.0,52.4,153.4,69.8,16.1,224.0,1.0,100.0,98.0,1.30,99.0,49.664343
2,A03,NIH1604001317,200611.0,2,7,47.0,50.6,153.0,70.2,15.0,177.0,1.0,106.0,100.0,1.10,103.0,59.931748
3,A04,NIH1604001317,200812.0,2,7,49.0,51.4,153.5,79.5,15.6,166.0,1.0,116.0,112.0,1.09,114.0,59.751741
4,A05,NIH1604001317,201007.0,2,7,51.0,52.2,153.0,71.4,15.0,189.0,1.0,110.0,112.0,1.03,111.0,63.092464
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,2,7,47.0,62.2,164.7,74.0,18.3,227.0,1.0,90.0,90.0,0.94,90.0,72.475151
16838,A04,NIH1604999643,200711.0,2,7,49.0,60.3,164.8,67.1,15.6,222.0,1.0,86.0,94.0,0.92,90.0,73.346554
16839,A05,NIH1604999643,200909.0,2,7,51.0,60.1,165.0,70.2,16.7,219.0,1.0,88.0,90.0,0.84,89.0,80.731689
16840,A06,NIH1604999643,201107.0,2,7,53.0,59.8,164.5,68.1,15.6,265.0,1.0,94.0,96.0,0.87,95.0,76.298749


In [15]:
final.loc[final['eGFR'] > 60, 'eGFR_60'] = 0
final.loc[final['eGFR'] <= 60, 'eGFR_60'] = 1

In [16]:
final

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED,SBP_AVG,eGFR,eGFR_60
0,A01,NIH1604001317,200209.0,2,7,43.0,50.0,153.5,70.0,14.3,209.0,1.0,88.0,90.0,1.00,89.0,69.167759,0.0
1,A02,NIH1604001317,200411.0,2,7,45.0,52.4,153.4,69.8,16.1,224.0,1.0,100.0,98.0,1.30,99.0,49.664343,1.0
2,A03,NIH1604001317,200611.0,2,7,47.0,50.6,153.0,70.2,15.0,177.0,1.0,106.0,100.0,1.10,103.0,59.931748,1.0
3,A04,NIH1604001317,200812.0,2,7,49.0,51.4,153.5,79.5,15.6,166.0,1.0,116.0,112.0,1.09,114.0,59.751741,1.0
4,A05,NIH1604001317,201007.0,2,7,51.0,52.2,153.0,71.4,15.0,189.0,1.0,110.0,112.0,1.03,111.0,63.092464,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,2,7,47.0,62.2,164.7,74.0,18.3,227.0,1.0,90.0,90.0,0.94,90.0,72.475151,0.0
16838,A04,NIH1604999643,200711.0,2,7,49.0,60.3,164.8,67.1,15.6,222.0,1.0,86.0,94.0,0.92,90.0,73.346554,0.0
16839,A05,NIH1604999643,200909.0,2,7,51.0,60.1,165.0,70.2,16.7,219.0,1.0,88.0,90.0,0.84,89.0,80.731689,0.0
16840,A06,NIH1604999643,201107.0,2,7,53.0,59.8,164.5,68.1,15.6,265.0,1.0,94.0,96.0,0.87,95.0,76.298749,0.0


In [ ]:
final.to_csv('eGFR_final.csv', encoding = 'euc-kr', index = False)

--------------------------

#### count CKD

In [9]:
df = pd.read_csv('eGFR_final.csv', encoding = 'euc-kr')
df

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED,SBP_AVG,eGFR,eGFR_60
0,A01,NIH1604001317,200209.0,2,7,43.0,50.0,153.5,70.0,14.3,209.0,1.0,88.0,90.0,1.00,89.0,69.167759,0.0
1,A02,NIH1604001317,200411.0,2,7,45.0,52.4,153.4,69.8,16.1,224.0,1.0,100.0,98.0,1.30,99.0,49.664343,1.0
2,A03,NIH1604001317,200611.0,2,7,47.0,50.6,153.0,70.2,15.0,177.0,1.0,106.0,100.0,1.10,103.0,59.931748,1.0
3,A04,NIH1604001317,200812.0,2,7,49.0,51.4,153.5,79.5,15.6,166.0,1.0,116.0,112.0,1.09,114.0,59.751741,1.0
4,A05,NIH1604001317,201007.0,2,7,51.0,52.2,153.0,71.4,15.0,189.0,1.0,110.0,112.0,1.03,111.0,63.092464,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,2,7,47.0,62.2,164.7,74.0,18.3,227.0,1.0,90.0,90.0,0.94,90.0,72.475151,0.0
16838,A04,NIH1604999643,200711.0,2,7,49.0,60.3,164.8,67.1,15.6,222.0,1.0,86.0,94.0,0.92,90.0,73.346554,0.0
16839,A05,NIH1604999643,200909.0,2,7,51.0,60.1,165.0,70.2,16.7,219.0,1.0,88.0,90.0,0.84,89.0,80.731689,0.0
16840,A06,NIH1604999643,201107.0,2,7,53.0,59.8,164.5,68.1,15.6,265.0,1.0,94.0,96.0,0.87,95.0,76.298749,0.0


In [10]:
unique_id = df['NIHID'].unique().tolist()

In [11]:
len(unique_id)

2406

In [12]:
df.loc[df['NIHID'] == 'NIH1604041378']

,기수,NIHID,EDATE,SEX,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,SMOKE,SBP_L,SBP_R,CREA_ADJUSTED,SBP_AVG,eGFR,eGFR_60
567,A01,NIH1604041378,200201.0,1,7,49.0,79.0,171.5,88.7,18.2,189.0,1.0,130.0,142.0,0.80,136.0,104.895287,0.0
568,A02,NIH1604041378,200308.0,1,7,50.0,77.5,171.0,90.0,16.9,203.0,1.0,102.0,126.0,1.10,114.0,77.860462,0.0
569,A03,NIH1604041378,200510.0,1,7,52.0,78.9,171.3,85.8,18.1,185.0,1.0,130.0,126.0,1.03,128.0,83.126410,0.0
570,A04,NIH1604041378,200709.0,1,7,54.0,79.7,170.5,86.0,18.8,195.0,1.0,122.0,122.0,1.13,122.0,73.280072,0.0
571,A05,NIH1604041378,200910.0,1,7,56.0,78.3,169.9,88.3,17.3,185.0,1.0,104.0,116.0,0.97,110.0,86.905887,0.0
572,A06,NIH1604041378,201107.0,1,7,58.0,78.8,169.8,86.4,17.9,214.0,1.0,124.0,120.0,1.16,122.0,69.027955,0.0
573,A07,NIH1604041378,201306.0,1,7,60.0,74.7,169.4,82.7,14.4,172.0,1.0,120.0,118.0,1.04,119.0,77.671181,0.0


In [21]:
CKD = 0
T = 0
for id in unique_id: 
    tmp = pd.DataFrame(df.loc[df['NIHID'] == id])
    tmp.reset_index(inplace = True, drop = True)
    count_healthy = 0
    count_unhealthy = 0
    for idx in range(len(tmp)): 
        if (tmp['기수'][idx] >= 'A01') and (tmp['기수'][idx] <= 'A04') and (tmp['eGFR_60'][idx] == 0): 
            count_healthy += 1
        if (tmp['기수'][idx] >= 'A05') and (tmp['기수'][idx] <= 'A07') and (tmp['eGFR_60'][idx] == 1): 
            count_unhealthy += 1
    print(id, count_healthy, count_unhealthy)

    if count_healthy == 4 and count_unhealthy == 3: 
        CKD += 1
    if count_healthy >= 2 and count_unhealthy == 3: 
        T += 1

NIH1604001317 1 2
NIH1604001328 4 0
NIH1604004884 4 0
NIH1604004989 4 0
NIH1604007653 4 0
NIH1604009453 4 1
NIH1604009656 4 0
NIH1604009672 4 0
NIH1604009888 4 0
NIH1604010156 4 0
NIH1604010641 4 0
NIH1604011703 4 0
NIH1604012036 4 0
NIH1604012038 4 0
NIH1604012392 4 0
NIH1604013399 4 1
NIH1604013954 4 0
NIH1604013996 4 0
NIH1604014047 3 1
NIH1604014533 4 0
NIH1604014630 4 0
NIH1604014650 4 2
NIH1604014685 4 0
NIH1604015999 4 0
NIH1604016292 4 2
NIH1604016354 4 0
NIH1604018504 3 3
NIH1604018564 4 0
NIH1604018586 4 0
NIH1604018695 4 0
NIH1604018748 4 0
NIH1604019385 3 3
NIH1604019544 4 0
NIH1604020059 4 0
NIH1604020120 4 0
NIH1604020196 4 0
NIH1604020318 4 2
NIH1604021425 4 0
NIH1604022010 4 0
NIH1604022443 4 0
NIH1604022500 2 3
NIH1604022722 4 0
NIH1604022976 4 0
NIH1604023000 4 0
NIH1604023904 4 0
NIH1604025057 4 0
NIH1604025108 4 1
NIH1604025445 3 0
NIH1604025601 4 1
NIH1604025691 0 3
NIH1604026104 4 1
NIH1604026465 4 0
NIH1604026629 4 0
NIH1604027423 3 2
NIH1604027816 4 0
NIH1604027

NIH1604217782 4 1
NIH1604218797 4 0
NIH1604219178 3 1
NIH1604219719 4 1
NIH1604222222 4 0
NIH1604223797 4 0
NIH1604223825 4 0
NIH1604224244 4 0
NIH1604225191 4 0
NIH1604225316 4 0
NIH1604225970 4 0
NIH1604226133 4 0
NIH1604226298 4 3
NIH1604226406 4 0
NIH1604226610 4 0
NIH1604227311 4 0
NIH1604227315 4 1
NIH1604228780 4 0
NIH1604228862 4 0
NIH1604228993 4 0
NIH1604229324 4 0
NIH1604229818 4 0
NIH1604229830 4 0
NIH1604230169 4 0
NIH1604230646 4 0
NIH1604230982 4 0
NIH1604231756 4 0
NIH1604231871 4 0
NIH1604232365 4 1
NIH1604233330 4 0
NIH1604234010 2 0
NIH1604234093 4 0
NIH1604234590 4 0
NIH1604235100 4 0
NIH1604235710 4 0
NIH1604236074 4 2
NIH1604236537 4 0
NIH1604237291 4 0
NIH1604238375 4 0
NIH1604238999 4 0
NIH1604239008 4 0
NIH1604239117 4 0
NIH1604240150 4 0
NIH1604240910 4 1
NIH1604241024 4 0
NIH1604241099 4 0
NIH1604242504 4 0
NIH1604242633 4 0
NIH1604243405 4 3
NIH1604243589 4 0
NIH1604243626 4 0
NIH1604243749 4 0
NIH1604243813 4 0
NIH1604244152 4 0
NIH1604244518 4 0
NIH1604244

NIH1604416713 4 0
NIH1604416920 4 0
NIH1604417421 4 0
NIH1604418154 4 0
NIH1604418932 4 0
NIH1604419064 4 0
NIH1604419461 4 0
NIH1604419750 4 0
NIH1604419883 4 0
NIH1604419927 4 0
NIH1604420098 4 0
NIH1604420437 3 2
NIH1604421031 4 0
NIH1604421582 4 1
NIH1604421613 4 0
NIH1604421784 4 0
NIH1604421790 4 0
NIH1604422471 4 0
NIH1604424415 4 0
NIH1604424659 4 0
NIH1604424764 4 0
NIH1604424806 4 2
NIH1604426748 4 0
NIH1604427347 4 0
NIH1604428311 4 0
NIH1604428498 4 1
NIH1604428713 4 0
NIH1604428854 4 0
NIH1604429119 4 0
NIH1604429207 4 0
NIH1604430105 4 0
NIH1604430235 4 0
NIH1604430970 4 0
NIH1604431083 4 0
NIH1604431102 4 1
NIH1604431175 4 0
NIH1604431854 4 1
NIH1604431994 4 0
NIH1604432047 4 0
NIH1604432259 4 0
NIH1604432426 4 0
NIH1604432727 4 0
NIH1604432986 4 1
NIH1604433294 4 0
NIH1604433326 4 0
NIH1604433344 2 2
NIH1604433379 4 0
NIH1604433384 4 0
NIH1604433799 4 0
NIH1604433827 4 0
NIH1604433853 4 1
NIH1604434047 4 2
NIH1604434983 4 1
NIH1604435651 4 0
NIH1604435674 4 2
NIH1604437

NIH1604631286 4 0
NIH1604631702 4 0
NIH1604631921 4 0
NIH1604632122 4 0
NIH1604632221 4 0
NIH1604632730 4 0
NIH1604632749 4 0
NIH1604633007 4 0
NIH1604633102 4 0
NIH1604634211 4 0
NIH1604634347 4 0
NIH1604634640 4 0
NIH1604634759 4 0
NIH1604634998 4 0
NIH1604635110 4 0
NIH1604635509 4 0
NIH1604636374 4 1
NIH1604636899 4 0
NIH1604637135 4 3
NIH1604637400 4 0
NIH1604637805 4 0
NIH1604637820 4 0
NIH1604638146 4 0
NIH1604638183 4 2
NIH1604638735 4 0
NIH1604639364 4 0
NIH1604639626 4 2
NIH1604640277 4 0
NIH1604640349 4 0
NIH1604640592 4 0
NIH1604641070 4 0
NIH1604641884 4 0
NIH1604642171 3 3
NIH1604643427 3 3
NIH1604643633 4 0
NIH1604643735 4 0
NIH1604644276 4 0
NIH1604644561 4 0
NIH1604646627 4 0
NIH1604646703 4 0
NIH1604646833 4 1
NIH1604647661 2 3
NIH1604647866 4 0
NIH1604648373 4 1
NIH1604648484 4 0
NIH1604648635 4 0
NIH1604648879 4 1
NIH1604648884 2 1
NIH1604648986 4 0
NIH1604650610 4 0
NIH1604651178 4 1
NIH1604651817 4 0
NIH1604651910 1 2
NIH1604652435 4 0
NIH1604652737 3 0
NIH1604653

NIH1604879053 4 0
NIH1604879650 4 0
NIH1604880654 4 0
NIH1604881091 4 0
NIH1604881313 4 0
NIH1604881442 1 2
NIH1604881560 4 0
NIH1604881862 4 2
NIH1604881872 4 0
NIH1604881940 4 0
NIH1604882155 4 0
NIH1604882513 4 0
NIH1604882932 4 0
NIH1604882960 4 3
NIH1604884941 4 0
NIH1604885495 3 0
NIH1604885621 3 2
NIH1604886353 4 0
NIH1604886404 4 0
NIH1604887241 4 0
NIH1604887441 4 0
NIH1604887521 4 0
NIH1604888703 4 0
NIH1604889407 4 0
NIH1604889550 4 0
NIH1604889604 4 0
NIH1604890268 4 0
NIH1604890935 4 2
NIH1604891635 4 0
NIH1604892017 4 0
NIH1604892481 4 0
NIH1604892594 4 0
NIH1604893078 4 0
NIH1604893548 4 0
NIH1604894122 4 0
NIH1604894597 4 0
NIH1604895868 4 0
NIH1604896288 4 0
NIH1604896587 4 0
NIH1604896643 4 0
NIH1604899097 4 0
NIH1604899591 4 0
NIH1604899735 4 1
NIH1604899958 3 0
NIH1604900097 4 0
NIH1604900516 4 0
NIH1604900536 4 0
NIH1604900931 4 0
NIH1604901346 4 0
NIH1604901677 4 0
NIH1604901699 4 0
NIH1604901957 4 0
NIH1604903005 4 0
NIH1604904415 4 0
NIH1604904703 3 3
NIH1604904

In [22]:
T

66

In [7]:
CKD

26

In [86]:
# dummy 변수화 전에 원래 변수 살려놓기 (stepwise용)
tt = pd.DataFrame(df[['SMOKE', 'SEX']])
tt

# dummy 변수 생성

dummy_cols = ['SMOKE', 'SEX']
df = pd.get_dummies(df, columns = dummy_cols)

df = pd.concat([df, tt], axis = 1)
df

,기수,NIHID,EDATE,VISITALL,AGE,WEIGHT,HEIGHT,WAIST,BODYFAT,TCHL_ORI,...,SBP_AVG,eGFR,eGFR_60,SMOKE_1.0,SMOKE_2.0,SMOKE_3.0,SEX_1,SEX_2,SMOKE,SEX
0,A01,NIH1604001317,200209.0,7,43.0,50.0,153.5,70.0,14.3,209.0,...,89.0,69.167759,0.0,1,0,0,0,1,1.0,2
1,A02,NIH1604001317,200411.0,7,45.0,52.4,153.4,69.8,16.1,224.0,...,99.0,49.664343,1.0,1,0,0,0,1,1.0,2
2,A03,NIH1604001317,200611.0,7,47.0,50.6,153.0,70.2,15.0,177.0,...,103.0,59.931748,1.0,1,0,0,0,1,1.0,2
3,A04,NIH1604001317,200812.0,7,49.0,51.4,153.5,79.5,15.6,166.0,...,114.0,59.751741,1.0,1,0,0,0,1,1.0,2
4,A05,NIH1604001317,201007.0,7,51.0,52.2,153.0,71.4,15.0,189.0,...,111.0,63.092464,0.0,1,0,0,0,1,1.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16837,A03,NIH1604999643,200601.0,7,47.0,62.2,164.7,74.0,18.3,227.0,...,90.0,72.475151,0.0,1,0,0,0,1,1.0,2
16838,A04,NIH1604999643,200711.0,7,49.0,60.3,164.8,67.1,15.6,222.0,...,90.0,73.346554,0.0,1,0,0,0,1,1.0,2
16839,A05,NIH1604999643,200909.0,7,51.0,60.1,165.0,70.2,16.7,219.0,...,89.0,80.731689,0.0,1,0,0,0,1,1.0,2
16840,A06,NIH1604999643,201107.0,7,53.0,59.8,164.5,68.1,15.6,265.0,...,95.0,76.298749,0.0,1,0,0,0,1,1.0,2


In [88]:
df.columns

Index(['기수', 'NIHID', 'EDATE', 'VISITALL', 'AGE', 'WEIGHT', 'HEIGHT', 'WAIST',
       'BODYFAT', 'TCHL_ORI', 'SBP_L', 'SBP_R', 'CREA_ADJUSTED', 'SBP_AVG',
       'eGFR', 'eGFR_60', 'SMOKE_1.0', 'SMOKE_2.0', 'SMOKE_3.0', 'SEX_1',
       'SEX_2', 'SMOKE', 'SEX'],
      dtype='object')

In [87]:
df.to_csv('eGFR_final_dummies.csv', encoding = 'euc-kr', index = False)

-----------------------------------------------------------------

#### sas용 dataset 생성

In [77]:
#sastest 생성
sastest = df.loc[(df['기수'] >= 'A01') & (df['기수'] <= 'A05')].reset_index(drop = True).copy()

In [78]:
sastest = sastest[['NIHID', 'EDATE', 'WEIGHT']].copy()

In [79]:
sastest['DATE'] = 0.0
sastest['YEAR'] = 0

In [80]:
sastest

,NIHID,EDATE,WEIGHT,DATE,YEAR
0,NIH1604001317,200209.0,50.0,0.0,0
1,NIH1604001317,200411.0,52.4,0.0,0
2,NIH1604001317,200611.0,50.6,0.0,0
3,NIH1604001317,200812.0,51.4,0.0,0
4,NIH1604001317,201007.0,52.2,0.0,0
...,...,...,...,...,...
12025,NIH1604999643,200109.0,61.0,0.0,0
12026,NIH1604999643,200305.0,67.5,0.0,0
12027,NIH1604999643,200601.0,62.2,0.0,0
12028,NIH1604999643,200711.0,60.3,0.0,0


In [81]:
for row in range(len(sastest)): 
    sastest['DATE'][row] = sastest['EDATE'][row]%100
    sastest['YEAR'][row] = int(sastest['EDATE'][row]/100)

sastest

<ipython-input-81-57048f248cde>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sastest['DATE'][row] = sastest['EDATE'][row]%100
<ipython-input-81-57048f248cde>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sastest['YEAR'][row] = int(sastest['EDATE'][row]/100)


,NIHID,EDATE,WEIGHT,DATE,YEAR
0,NIH1604001317,200209.0,50.0,9.0,2002
1,NIH1604001317,200411.0,52.4,11.0,2004
2,NIH1604001317,200611.0,50.6,11.0,2006
3,NIH1604001317,200812.0,51.4,12.0,2008
4,NIH1604001317,201007.0,52.2,7.0,2010
...,...,...,...,...,...
12025,NIH1604999643,200109.0,61.0,9.0,2001
12026,NIH1604999643,200305.0,67.5,5.0,2003
12027,NIH1604999643,200601.0,62.2,1.0,2006
12028,NIH1604999643,200711.0,60.3,11.0,2007


In [82]:
sastest.rename(columns = {'NIHID' : 'ID', 
                         'YEAR' : 'year', 
                         'DATE' : 'date', 
                         'WEIGHT' : 'weight'}, inplace = True)

In [83]:
sastest.drop(['EDATE'], axis = 1, inplace = True)

In [84]:
sastest

,ID,weight,date,year
0,NIH1604001317,50.0,9.0,2002
1,NIH1604001317,52.4,11.0,2004
2,NIH1604001317,50.6,11.0,2006
3,NIH1604001317,51.4,12.0,2008
4,NIH1604001317,52.2,7.0,2010
...,...,...,...,...
12025,NIH1604999643,61.0,9.0,2001
12026,NIH1604999643,67.5,5.0,2003
12027,NIH1604999643,62.2,1.0,2006
12028,NIH1604999643,60.3,11.0,2007


In [85]:
sastest.to_csv('sastest1013.csv', encoding = 'euc-kr', index = False)